# **Washoe County 2017 Sales Report Analysis**

This is analysis, we will look into the [Washoe County Sales Report Data](https://www.washoecounty.us/assessor/online_data/sales_reports.php). This is a dataset that includes data on sales made in Washoe County during the 2017 fiscal year. We will clean this data up and use it to get an insight into the housing market in Washoe County. We can also apply regression to this dataset and predict housing prices based on different features included in the Sales Report.

![](https://cdnparap100.paragonrels.com/ParagonImages/Property/P10/NNRMLS/170012273/0/0/0/662321a8ae87b96b5ef6e001d90e839e/15/2301c3cce54ff5e8c05025ff3c970831/170012273.JPG)

## **Import Libraries**

We start by importing the necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

## **Read in data**

We now read in our data from the Washoe County website given in xls format. 
* [Washoe Sales Report Link](https://www.washoecounty.us/assessor/online_data/sales_reports.php)

In [2]:
sales = pd.read_excel('RDEQISales2017.xls',sheet_name='Sheet1',header=1,na_values=np.nan)

FileNotFoundError: [Errno 2] No such file or directory: 'RDEQISales2017.xls'

Let's look at what was read in.

In [ ]:
sales.head()

Now, let's extract some basic information from our dataset.

In [ ]:
sales.describe()

In [ ]:
sales.info()

## **Let's get a better understanding of our data!**

Here I am just exploring the dataset a bit, I was interested in the APN column and wanted to know more about it.

In [ ]:
sales[sales['APN'] == sales['APN'].value_counts().index[0]]

As we can see it seems to be a type of building code that represents certain buildings in Washoe County.

Since we are interested in analyzing sales of homes, we must only consider data that has a building type and address entries.

We should look at the different type of buildings to see the variations.

In [ ]:
build_type = sales['BldgType'].value_counts().index
sales['BldgType'].count()

We have started with 13,500 data entries and only have building types for 12,047 of them.

In [ ]:
build_type

In [ ]:
print('Data Entries: {}, Percent of data without a street address or type of building or stories entries entries: {:.2f}%'.format(
    len(sales[(sales['BldgType'].isna()) | (sales['Situs'].isna()) | (sales['Stories'].isna())]),
    len(sales[(sales['BldgType'].isna()) | (sales['Situs'].isna() | (sales['Stories'].isna()))])
    /len(sales)*100))

messy = sales[(sales['BldgType'].notna()) & (sales['Situs'].notna()) & (sales['Stories'].notna())].copy()

Here we can see that we essentially have lost 14% of the data from the original sales report as they do not have a address, building type, or stories entries.

Let's now dig deeper into the data. Let's check out our dataframe once again!

In [ ]:
messy.info()

We now have 11,618 entries from 13,500 original entries. Let's look at the value count of the 'BldgType' column.

In [ ]:
messy['BldgType'].value_counts()

As we  see there are 63 different values in the column. This is way too many and we can shrink this down a little and aggregate some entries together, so let's do that.

## **Cleaning our Data**

Let's build a dictionary of replacement values.

In [ ]:
building_type_dict = {'Sgl Fam Res ':'Residential House', 'Townhse End':'End Townhouse','Mixed Retail w/ Resid. Units':'Store',
                      'Townhse Ins':'Middle Townhouse', 'Multiple Res (Low Rise)':'Low Rise Apartment',
                      'HiRise Condo':'High Rise Condo', 'Parking Level':'Parking',
                      'Industrial Flex Building':'Industrial Building', 'Retail Store':'Store',
                     'Discount Store':'Store', 'Service Repair Garage':'Repair Garage',
                      'Hotel, Limited Service':'Hotel', 'Hotel Condo':'Hotel',
                     'Neighborhood Shopping Ctr':'Shopping Center','Fast Food Restaurant':'Restaurant',
                     'Mini-Warehouse':'Warehouse','Mega Warehouse':'Warehouse',
                      'Equipment (Shop) Building':'Equipment Shop','Hotel, Full Service':'Hotel',
                     'Convenience Market':'Store','Industrials, Light Mftg.':'Industrial Building',
                     'Shell, Office':'Office Building','Regional Shopping Center':'Shopping Center',
                     'Dental Office/Clinic':'Medical Building','Mini-Lube Garage':'Repair Garage',
                     'Apartment':'Low Rise Apartment','Lt. Commercial Utility Build.':'Industrial Building',
                     'Community Shopping Center':'Shopping Center','Multiple Res. (Sen. Citizen)':'Group Care Home',
                     'Theater - Live Stage':'Entertainment Building','Automotive Center':'Car Dealership',
                     'Market':'Store','Veterinary Hospital':'Medical Building',
                     'Distribution Warehouse':'Warehouse','Clubhouse':'Country Club',
                     'Storage Garage':'Storage','Storage Warehouse':'Storage',
                     'Supermarket':'Store','Automobile Showroom':'Entertainment Building',
                     'Fire Station (Volunteer)':'Fire Station','Bowling Center':'Entertainment Building',
                     'Shell, Neigh. Shop. Ctr.':'Shopping Center','Service Garage Shed':'Repair Garage',
                     'Shed Office Structure':'Office Building','Handball-Racquetball Club':'Country Club',
                     'Discount Warehouse Store':'Store','Medical Office':'Medical Building',
                     'MH Real Prop':'MH Real Prop','Office Building':'Office Building',
                     'Duplex':'Duplex','Conversion':'Conversion','Restaurant':'Restaurant',
                     'Motel':'Motel','Bar/Tavern':'Bar','Casino':'Entertainment Building','Bank':'Bank',
                     'Church':'Church','Barber Shop':'Barber Shop','Day Care Center':'Day Care Center',
                     'Group Care Home':'Group Care Home','Country Club':'Country Club',
                     'Restroom Building':'Restroom Building','Classroom':'Classroom','Dispensary':'Dispensary'}




In [ ]:
messy['BldgType'] = messy['BldgType'].map(building_type_dict)

We have now mapped the dictionary to our column, let's look at what it did!

In [ ]:
messy['BldgType'].value_counts()

In [ ]:
messy['BldgType'].count()

Perfect, we changed all the column values and ended with how many we started with!

Let's now create new columns based off previous columns. This is a bit of feature engineering that will help us better visualize our data and extract more useful information.

We will make a 'month','day' and 'street' column based off the 'Sales Date' and 'Situs' columns. We also go about dropping some columns that are not necessary to our analysis at the moment and are full of null values.

In [ ]:
messy['Sale Month'] = messy['Sales Date'].apply(lambda x: int(x.split('/')[0]))
messy['Sale Day'] = messy['Sales Date'].apply(lambda x: int(x.split('/')[1]))
messy['Street'] = messy['Situs'].apply(lambda x: ' '.join(x.split()[1:]))

#While we are at it let's also drop the columns Add Rec, Bsmt Type, Mailing2
cleaned = messy.drop(['Situs','Sales Date' ,'Add Rec', 'Bsmt Type', 'Mailing2'],axis=1).copy()

cleaned.head()

In [ ]:
cleaned.info()

In [ ]:
cleaned.describe()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(cleaned.isnull())

Here we can see we are very close to cleaning our data entirely for our analysis. We will export this cleaner version of our data to a csv.

In [ ]:
cleaned.to_csv('Cleaned_WashoeSalesReport2017.csv',index=False)

Before we go on and choose a subset of the data for our analysis, we should notice a interesting feature of our data.

By looking at the different bins of the sales price on residential houses we can see how the data blows up at certain higher million dollar sales prices.

In [ ]:
# Notice weird trend in data
total_residential = cleaned[(cleaned['BldgType'] == 'Residential House')]

house_less_than = [cleaned[(cleaned['BldgType'] == 'Residential House') 
                           & (cleaned['Sale Price'] < 1e6)],
                   cleaned[(cleaned['BldgType'] == 'Residential House') 
                           & (cleaned['Sale Price'] < 5e6)],
                   cleaned[(cleaned['BldgType'] == 'Residential House') 
                           & (cleaned['Sale Price'] < 10e6)],
                   cleaned[(cleaned['BldgType'] == 'Residential House') 
                           & (cleaned['Sale Price'] < 15e6)],
                   cleaned[(cleaned['BldgType'] == 'Residential House') 
                           & (cleaned['Sale Price'] < 18e6)],
                   cleaned[(cleaned['BldgType'] == 'Residential House') 
                           & (cleaned['Sale Price'] < 19e6)]]

In [ ]:
list_res = [1,5,10,15,18,19]

for price,data in list_res,house_less_than:
    print('Houses less than {} million: {}'.format(price,len(data)))

Notice we get a reasonable change from 1-5 million. We get a total of 77 houses sold from 5-18 million and a total of 3 sold from 10-18 million. Very oddly, we now notice an increase of 78 houses between 18-19 million! This was odd to see and I do not know what caused this in the data. If I had to make a guess I would say it has something to do with housing developers buying large quantities of houses at once, but I cannot be exactly sure and it is fairly unimportant to this particular analysis since we will be dealing with houses under a million dollars.

Though it will not affect our analysis, it is a very interesting trend to see in something like the Washoe County Sales Report since this represents some type of real event!

## **Choosing a subset of our data**

Now that we have cleaned our data up so that it can be useful, we may now take the subset we are interested in.

In [ ]:
house_more_mil = cleaned[(cleaned['BldgType'] == 'Residential House') 
                         & (cleaned['Sale Price'] >= 1e6)].copy()

percent = len(house_more_mil)/len(total_residential)

print('Percentage of residential houses over 1 million dollars: {:.2f}%'.format(percent*100))

We can see that we have are essentially going to leave out about 7% of our cleaned data.

Here we grab the residential houses below or equal to a million dollars.

In [ ]:
residential_house = cleaned[(cleaned['BldgType'] == 'Residential House') 
                            & (cleaned['Sale Price'] < 1e6)].copy()

In [ ]:
residential_house.info()

In [ ]:
residential_house.describe()

## **EDA**

Now we may start to analyze the data on residential homes in Washoe County. Let's see what the minimum and maximum price of a residential house sold in Washoe County was in 2017.

In [ ]:
house_price_min,house_price_max = (residential_house['Sale Price'].min(),
                                   residential_house['Sale Price'].max())

print('Min: {} , Max: {} '.format(house_price_min,house_price_max))

Wow, $27,019! Very interesting, let's look at that entry.

In [ ]:
residential_house[residential_house['Sale Price'] 
                  == residential_house['Sale Price'].min()]

Seems like a reasonable entry, but is no doubt an odd/interesting one.

## **Visualizations**

We now want to answer some questions about our data by using some visualizations.

We are interested in answering questions such as,
* How much does a house cost in Washoe County?
* What months are most popular for houses to be sold in?
* Where are homebuyers moving in from?
* How much square footage in a house can a person expect in Washoe County?

In [ ]:
bins = [0,1e5,2e5,3e5,4e5,5e5,6e5,7e5,8e5,9e5,10e5]

norm_sales = pd.cut(residential_house['Sale Price'],
                    bins).value_counts(sort=False,
                                       normalize=True)

In [ ]:
norm_sales.plot.bar(rot = 90,color="r", figsize=(12,8))
plt.title('Normalized Bar Chart of House prices in Reno, Nevada')
plt.ylabel('Percentage of distribution')
plt.xlabel('Price Intervals')


We can see that more than 70% of houses are sold between $200-500k in Washoe County!

Let's now look at the most popular month houses are sold in!

In [ ]:
sales_by_month = residential_house['Sale Month'].value_counts(sort=True)

In [ ]:
sales_by_month = sales_by_month.sort_index()

sales_by_month = sales_by_month.rename({
    1:'January',
    2:'February',
    3:'March',
    4:'April',
    5:'May',
    6:'June',
    7:'July',
    8:'August',
    9:'September',
    10:'October',
    11:'November',
    12:'December',})

In [ ]:
sales_by_month.plot.bar(color='r',figsize=(12,8))
plt.title('Sales of Residential Houses by Month')
plt.ylabel('Number of Houses Sold')
plt.xlabel('Months')

Here we can see that the distribution looks relatively normal, but with a slight right skew. From this we can confidently say that houses are sold less in the first 4 months of the year than the last 8 months, with the summer months between June-August being the most popular and a steady decline of houses sold from August-December.

We also can make a heatmap of the months to get help us get a better understanding of the months in comparison with each other.

In [ ]:
plt.figure(figsize=(12,8))

sns.heatmap(sales_by_month.to_frame('Houses Sold').transpose(),
            annot=True,cmap='magma',fmt='.2f')

plt.title('Heatmap of Months vs. Houses Sold')
plt.ylabel('Months')

We may also want to know what type of home buyers we are dealing with, so let's see where our home buyers are from!

In [ ]:
fig,axes = plt.subplots(figsize=(12,8))

sns.barplot(x=residential_house['State'].value_counts(normalize=True).head(5).values,
            y=residential_house['State'].value_counts(normalize=True).index[:5])

plt.title('Normalized Bar Chart of States Home Buyers are from')
plt.xlabel('Percentage of Home Buyers')
plt.ylabel('States')

Surprisingly, it looks like more than 90% are from Nevada while less than 10% are from CA.

Lastly, let's look at what the general square footage of a house in Washoe County looks like just to help us visualize some more of our data.

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(residential_house['Bldg SF'],color='r')
plt.title('Distribution Plot of Square Feet in Sold Houses')
plt.xlabel('House Square Footage')

This is informative, but since the 'Year Blt' column is also available maybe we can extract some more useful information.

We should start by binning our values and then we can use the groupby method to see some of the different values based on binned years.

By binning the year built column we can now continue to answer more questions such as,
* Does buying a more recently built home have an affect on the size of the house?
* Do more recently built homes tend to cost more than older homes and by how much?

In [ ]:
bins = [1900,1920,1940,1960,1970,1980,1990,2000,2010,2017]
residential_house['Year bins'] = pd.cut(residential_house['Year Blt'],bins)
residential_house['Year bins'].value_counts(sort=False)

In [ ]:
average_sq_ft = residential_house[['Bldg SF','Year bins']].groupby('Year bins').mean()

fig,axes = plt.subplots(1,1,figsize=(12,8))

sns.barplot(x=average_sq_ft.index.astype('str'),
            y=average_sq_ft.values.reshape(len(average_sq_ft)),
            ax=axes)

plt.xlabel('Bins of Years')
plt.ylabel('Average Square Footage')
plt.title('Average Square Footage in Houses Sold in 2017 given by Binned Years')

As we can see here, this graph shows an increase in square footage as the years progress and also gives a good representation of the amount of space you could expect in a house built during a certain time.

Let's do the same for the Sales Price.

In [ ]:
average_sq_ft = residential_house[['Sale Price','Year bins']].groupby('Year bins').mean()

fig,axes = plt.subplots(1,1,figsize=(12,8))

sns.barplot(x=average_sq_ft.index.astype('str'),
            y=average_sq_ft.values.reshape(len(average_sq_ft)),
            ax=axes)

plt.xlabel('Bins of Years')
plt.ylabel('Average Sales Price')
plt.title('Average Sales Price of Houses Sold in 2017 given by Binned Years')

Here we also see that you could generally expect to pay more for a house built more recently and less for an older home. Interestingly enough, we see a rise in home prices from 1920-1940, what this is accounted to is unknown to me. We could possibly account this to Washoe County being rather historical and the time between 1920-1940 being a special time in history, thus causing homes from that time period to cost more.

## **Preparing data for prediction**

Now that we have visualized our data let's go ahead and try to make predictions for our data on the Sales Price.

We must first make sure our data is all in numeric form so that the models can be fit properly.

We will start by looking at two of our columns 'Stories' and 'Grade'. We want to group similar values together so that the model can predict better and not have data values with only a few entries.

In [ ]:
residential_house['Grade'].value_counts()

In [ ]:
residential_house['Grade'] = residential_house['Grade'].replace({'7.0 HIGH VALUE CLASS I':'Average',
                                                                 '8.5 HIGH VALUE CLASS II/HIGH VALUE CLASS III':'Good-Very Good',
                                                                 '9.0 HIGH VALUE CLASS III':'Very Good'},inplace=True)

In [ ]:
residential_house['Grade'].value_counts()

Here we have simply replaced the 3 values with better corresponding groups.

Now we will perform the same for the 'Stories' column.

In [ ]:
residential_house['Stories'].value_counts()

In [ ]:
residential_house['Stories'].replace({'1.5 STRY FN':'SINGLE HALF STORY',
                                     '2.5 STRY FN':'TWO HALF STORY',
                                     '1.5 STRY UNF':'SINGLE HALF STORY',
                                     'BI_LEVEL':'SPLIT LEVEL'},inplace=True)

In [ ]:
residential_house['Stories'].value_counts()

And very similarly, we now have better grouped columns for our predictor values.

Now let's look at our data once more to see what columns we do not need for our predictions.

In [ ]:
residential_house.info()

We will drop a lot of these columns for predictions since they would not be very helpful to predict a Sales Price.

In [ ]:
drop = [
    'APN',
    'Sale Verf',
    'RecDoc',
    'Subdivision',
    'Avg Yr Blt',
    'Total Units',
    'TaxDist',
    'LegalDesc',
    'PriorPID',
    'PriorOwner',
    'Zip',
    'State',
    'City',
    'Mailing1',
    'Addl Owner',
    'Owner1',
    'Zoning',
    'LUC at Sale',
    'BldgType',
    'Year bins'
]

residential_house.drop(drop,axis=1,inplace=True)

Here we have dropped our columns and are ready to proceed to making our predictions!

## **Making Predictions**

Let's first import our library functions from sklearn.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,LabelEncoder 
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE

Let's look at what our current predictors looks like.

In [ ]:
residential_house.info()

We will define three of our methods here,
* LabelEncoder() - to encode values numerically
* StandardScaler() - to standardize the data for RFE
* LinearRegression() - to predict our Sales Price

In [ ]:
le = LabelEncoder()
sc = StandardScaler()
lm = LinearRegression()

Let's use our Label encoder to encode the 'Grade' and 'Stories' columns we cleaned up earlier.

In [ ]:
residential_house['Grade'] = le.fit_transform(residential_house['Grade'])
residential_house['Stories'] = le.fit_transform(residential_house['Stories'])

Now that we have encoded the string columns we wanted, let's get rid of the rest of the categorical data.

In [ ]:
categorical = [columns 
               for columns 
               in residential_house.dtypes.index 
               if residential_house.dtypes[columns] == 'object']

residential_house.drop(categorical,axis=1,inplace=True)

Now we must define our X and Y, Y will be assigned to the 'Sale Price' column and X will be the rest of the columns we prepared.

We will also save the name of the columns and then standardize our data for RFE.

RFE stands for Recursive Feature Elimination, using this we may assign weights to our features and learn which is the most important towards our prediction.

In [ ]:
y = residential_house['Sale Price']
X = residential_house.drop(['Sale Price'],axis=1)

old_columns = X.columns
X = sc.fit_transform(X)
X = pd.DataFrame(X,columns=old_columns)

# We tell RFE to use LinearRegression as the model, rank 4 features the best, and only dispose of 1 
# feature at a time each iteration
selector = RFE(lm,4,1)
selector = selector.fit(X,y)

Here we may now look at the rankings in order of column position.

In [ ]:
selector.ranking_

We only are interested in the 4 best columns, so we can drop all the rest using list comprehension and the drop function.

In [ ]:
columns_not_one = [i 
                   for (i,x) 
                   in enumerate(selector.ranking_) 
                   if (x != 1)]

print(columns_not_one)

X.drop(X.columns[columns_not_one],axis=1,inplace=True)

In [ ]:
X.columns

We may now see what columns are left as predictors for Sales Price.

Let's just look at the data and make sure everything is okay.

In [ ]:
sns.jointplot(X)

In [ ]:
X.shape

In [ ]:
X.head()

Now we must split our data into two sets: a train set and a test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=99)

We can now fit our model using the training set and training y values.

In [ ]:
lm.fit(X_train,y_train)

Now that our model is trained we can use the test set as 'new data' and test the accuracy of our predictions.

In [ ]:
predictions = lm.predict(X_test)

Using and R_squared score we may look at the accuracy between our predicted values and the actual values.

In [ ]:
print(r2_score(y_test,predictions))

We see that our predictions are at 58%, this is not bad, but not great either. We could improve this by adding more features, possibly incoporating market trends for housing, etc.

Lastly, let's plot our residuals to make sure it is normal and centered around 0.

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(predictions-y_test,color='r')

Great! We have residuals that look normal and seem to be very close to a mean of 0 indicating a good linear model fit. 